# Dependencies

In [1]:
from ast import parse
import os
from diffusers import StableDiffusionPipeline
from tqdm import tqdm
import itertools
from datetime import datetime
import argparse
# from ml_collections import ConfigDict
import torch
import random
import numpy as np
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
from PIL import Image

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# Configuration -- Set Parameters here

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True
)
pipe = pipe.to(device)


def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Configuration dict
config = {
    "global_save_label": "beach_only",
    "batch_size": 100,
    "minibatch_size": 2,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "seed": 1,
    "num_iters": 1,
    "machine_name": "default",
}

# Extract config values
global_save_label = config["global_save_label"]
batch_size = config["batch_size"]
minibatch_size = config["minibatch_size"]
device = config["device"]
seed = config["seed"]
num_iters = config["num_iters"]
machine_name = config["machine_name"]

torch_device = torch.device(device)

set_seed(seed)


/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_loading_utils.py:221: FutureWarning: You are loading the variant fp16 from CompVis/stable-diffusion-v1-4 via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(
Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--

# Source Code

In [9]:

# Generate Batch Function
def generate_batch(prompt: str, save_label: str, keywords: list = ['beach'], negative_prompt: str = "human, blurry, painting, cartoon, extra limbs, disfigured, deformed, body out of frame, bad anatomy, watermark, two, multiple", num_inference_steps: int = 50, batch_size: int = 80, minibatch_size: int = 2, additional_label: str = None):
    """
    Generate a batch of images from a prompt.
    Save the images to a folder specified by the save label.
    """
    print('Generating images with prompt: \n', prompt)
    os.makedirs(save_label, exist_ok=True)

    # Calculate the start index based on existing images
    existing_images = [f for f in os.listdir(save_label) if f.endswith(".png")]
    start_index = len(existing_images)

    prompt_list = [prompt] * minibatch_size
    negative_prompt_list = [negative_prompt] * minibatch_size
    batch_count = 0

    with tqdm(total=batch_size) as pbar:
        while batch_count < batch_size:
            # Generate a batch of images
            output = pipe(
                prompt_list,
                negative_prompt=negative_prompt_list,
                num_inference_steps=num_inference_steps,
            )
            images = output.images

            # Save the images
            for idx, image in enumerate(images):
                save_path = os.path.join(save_label, f"{start_index + batch_count + idx}.png")
                if additional_label is not None:
                    save_path = os.path.join(save_label, f"{additional_label}_{start_index + batch_count + idx}.png")
                image.save(save_path, format="png")
                pbar.update(1)

            batch_count += len(images)

    print(f"{batch_count} images generated and saved to {save_label}")



# Run the image generation!

In [3]:
locations = ["desert","jungle", "dirt", "snow"]
location_dict = {
    'jungle': 'in a jungle',
    'mountain': 'on a rocky mountain',
    'desert': 'in a hot, dry desert with cactuses around',
    'dirt': 'in a park, with puddles, bushes and dirt in the background',
    'beach': 'playing fetch on a beach with a pier and ocean in the background',
    'snow': 'in a snowy landscape with a cabin and a snowball in the background',
}

In [11]:
for location in locations:
    location_phrase = location_dict[location]

    prompt = f"{location_phrase}, highly detailed, with cinematic lighting, 4k resolution, beautiful composition, hyperrealistic, trending, cinematic, masterpiece, close up"
    label = f"./generated_datasets/background_only_dataset/{location}_only"

    print(f"Generating images for : {location}; prompt: {prompt}")

    generate_batch(
        prompt=prompt,
        save_label=label,
        keywords=[location],
        batch_size=batch_size,
        minibatch_size=minibatch_size
    )

Generating images for : desert; prompt: in a hot, dry desert with cactuses around, highly detailed, with cinematic lighting, 4k resolution, beautiful composition, hyperrealistic, trending, cinematic, masterpiece, close up
Generating images with prompt: 
 in a hot, dry desert with cactuses around, highly detailed, with cinematic lighting, 4k resolution, beautiful composition, hyperrealistic, trending, cinematic, masterpiece, close up


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  1%|          | 1/100 [00:02<04:37,  2.80s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

 97%|█████████▋| 97/100 [02:22<00:04,  1.52s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 100/100 [02:25<00:00,  1.45s/it]

100 images generated and saved to ./generated_datasets/background_only_dataset/snow_only


# Zip & Move the files into our dataset_dir

In [16]:
import os
import tarfile
from google.colab import drive
from google.colab import files

# Function to tar and compress a dataset
def tar_and_upload(dataset_path, output_tar_name, drive_folder_path):
    """
    Tar a dataset into a .tar.gz file and upload it to Google Drive in Colab.

    Parameters:
    - dataset_path (str): Path to the dataset directory to be tarred.
    - output_tar_name (str): Name of the resulting .tar.gz file.
    - drive_folder_path (str): Path to the folder in Google Drive where the file will be saved.
    """
    # Mount Google Drive
    drive.mount('/content/drive')

    # Check if the dataset path exists
    if not os.path.exists(dataset_path):
        print(f"Dataset path {dataset_path} does not exist.")
        return

    # Create the .tar.gz file
    tar_path = f"/content/{output_tar_name}"
    with tarfile.open(tar_path, "w:gz") as tar:
        tar.add(dataset_path, arcname=os.path.basename(dataset_path))
    print(f"Created tar.gz file at: {tar_path}")

    # Ensure the Drive folder exists
    drive_full_path = os.path.join('/content/drive/My Drive', drive_folder_path)
    os.makedirs(drive_full_path, exist_ok=True)

    # Move the tar.gz file to the specified folder on Drive
    target_path = os.path.join(drive_full_path, output_tar_name)
    !cp $tar_path $drive_folder_path
    print(f"File moved to Google Drive at: {target_path}")


In [17]:
dataset_name = "background_only_dataset"
project_dir = "/content/drive/MyDrive/CS2822_Final_Project"
datasets_dir = os.path.join(project_dir, "Datasets")

local_data_dir = os.path.join("/content/generated_datasets", dataset_name)
drive_dest_dir = os.path.join(datasets_dir, dataset_name+".tar.gz")

# Tar and copy the dataset into drive
tar_and_upload(local_data_dir, dataset_name+".tar.gz", datasets_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created tar.gz file at: /content/background_only_dataset.tar.gz
File moved to Google Drive at: /content/drive/MyDrive/CS2822_Final_Project/Datasets/background_only_dataset.tar.gz
